In [ ]:
%matplotlib inline
import multiprocessing

import IPython.display as ipd
import torch
from torch.utils.data import DataLoader

from hparams import get_hparams_from_file
from src.data.collate import TextAudioSpeakerCollate
from src.data.dataset import TextAudioSpeakerDataset
from src.model.checkpoint import load_checkpoint
from src.model.synthesizer import SynthesizerTrn
from src.text.convert import preprocess_text
from src.text.symbols import symbols

## VCTK (Multispeaker)

In [ ]:
hps = get_hparams_from_file("./configs/vctk_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = load_checkpoint(hps.train, "/path/to/pretrained_vctk.pth", net_g, None)

In [ ]:
stn_tst = preprocess_text("VITS is Awesome!", hps.data.text_cleaners)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([4]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][
        0, 0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sample_rate, normalize=False))


### Voice Conversion


In [ ]:
dataset = TextAudioSpeakerDataset(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
num_cpus = multiprocessing.cpu_count()
loader = DataLoader(dataset, num_workers=num_cpus, shuffle=False,
                    batch_size=1, pin_memory=True,
                    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][
        0, 0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][
        0, 0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][
        0, 0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sample_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sample_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sample_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sample_rate, normalize=False))
